In [9]:
import pandas as pd
import re

# Load original data
df = pd.read_csv("Multiple-choice Question with a Single Correct Answer.csv")

# Cleaning function
def extract_mcq(row):
    qtext = row['question']
    prompt = row['question_prompt']

    # Extract main question
    parts = re.split(r'\nA\.', qtext, maxsplit=1)
    question_main = parts[0].strip()
    
    # Extract A–D
    options_raw = 'A.' + parts[1] if len(parts) > 1 else ''
    pattern = r'([A-D])\.\s(.*?)(?=\n[A-D]\.|\Z)'
    options = dict(re.findall(pattern, options_raw, re.DOTALL))
    
    # Extract E and remove from prompt
    prompt_lines = prompt.strip().split('\n')
    option_e = ''
    if prompt_lines and prompt_lines[0].strip().startswith("E."):
        option_e = prompt_lines[0][2:].strip()
        cleaned_prompt = "\n".join(prompt_lines[1:]).strip()
    else:
        cleaned_prompt = prompt

    return pd.Series({
        'question': question_main,
        'question_prompt': cleaned_prompt,
        'A': options.get('A', ''),
        'B': options.get('B', ''),
        'C': options.get('C', ''),
        'D': options.get('D', ''),
        'E': option_e
    })

# Apply
extracted_df = df.apply(extract_mcq, axis=1)

# Merge cleaned fields
final_df = pd.concat([
    df[['qid', 'video_id', 'youtube_url', 'duration', 'question_type', 'capability']],
    extracted_df
], axis=1)

# Save
final_df.to_csv("mcq_cleaned.csv", index=False)

print("✅ Done! Saved as mcq_cleaned.csv")

✅ Done! Saved as mcq_cleaned.csv
